#### 엑손 영역에 대한 가상의 유전변이를 생성하도록 하자.

In [1]:
#import exon region
# BRCA1 exon 10 region RefSeqBRCA.tsv
'''
Chr	Start	Stop	Gene Name	Transcript Name	Type	Transcript Type	Strand	CDS Start	CDS Stop	Exon Starts	Exon Stops	Product ID	Prote
in ID	GeneID	HGNC	MIM	CCDS	HPRD	Summary of Product	LRG ID
17	41196312	41277500	BRCA1	NM_007300.3	mRNA	mRNA	-	41197694	41276113	41243451	41246877	NP_009231.2	NP_00
9231.2	672	1100	113705	CCDS11456.2	?	breast cancer type 1 susceptibility protein isoform 2	?
'''
import itertools
import random
import os
import datetime
import pandas as pd
import numpy as np
from HTSeq import FastaReader

raw_df=pd.read_csv("RefSeqBRC_2.tsv", sep="\t")

In [2]:
raw_df

,Chr,Start,Stop,Gene Name,Transcript Name,Type,Transcript Type,Strand,CDS Start,CDS Stop,...,Exon Stops,Product ID,Protein ID,GeneID,HGNC,MIM,CCDS,HPRD,Summary of Product,LRG ID
0,17,41196312,41277500,BRCA1,NM_007294.3,mRNA,mRNA,-,41197694,41276113,...,"41197819,41199720,41201211,41203134,41209152,4...",NP_009225.1,NP_009225.1,672,1100,113705,CCDS11453.1,?,breast cancer type 1 susceptibility protein is...,LRG_292t1


In [3]:
raw_df["Exon Starts"][0]

'41196311,41199659,41201137,41203079,41209068,41215349,41215890,41219624,41222944,41226347,41228504,41234420,41242960,41243451,41247862,41249260,41251791,41256138,41256884,41258472,41267742,41276033,41277287'

In [4]:
tx_exon_dict=dict()
for i in range(len(raw_df)):
    exon_start_stop = zip(map(int,raw_df["Exon Starts"][i].split(",")),map(int, raw_df["Exon Stops"][i].split(",")))
    tx_exon_dict[raw_df["Transcript Name"][i]]= exon_start_stop

In [5]:
for tx_name, exon_coordinate_list in tx_exon_dict.items():
    print tx_name
    print exon_coordinate_list

NM_007294.3
[(41196311, 41197819), (41199659, 41199720), (41201137, 41201211), (41203079, 41203134), (41209068, 41209152), (41215349, 41215390), (41215890, 41215968), (41219624, 41219712), (41222944, 41223255), (41226347, 41226538), (41228504, 41228631), (41234420, 41234592), (41242960, 41243049), (41243451, 41246877), (41247862, 41247939), (41249260, 41249306), (41251791, 41251897), (41256138, 41256278), (41256884, 41256973), (41258472, 41258550), (41267742, 41267796), (41276033, 41276132), (41277287, 41277500)]


In [6]:
expanded_tx_exon_dict=dict()
for tx_name, exon_coordinate_list in tx_exon_dict.items():
    new_coordiante_list=[]
    for start, stop in exon_coordinate_list:
        new_start=start-100
        new_stop=stop+100
        new_coordiante_list.append((new_start, new_stop))
    expanded_tx_exon_dict[tx_name] = new_coordiante_list
    

In [7]:
expanded_tx_exon_dict

{'NM_007294.3': [(41196211, 41197919),
  (41199559, 41199820),
  (41201037, 41201311),
  (41202979, 41203234),
  (41208968, 41209252),
  (41215249, 41215490),
  (41215790, 41216068),
  (41219524, 41219812),
  (41222844, 41223355),
  (41226247, 41226638),
  (41228404, 41228731),
  (41234320, 41234692),
  (41242860, 41243149),
  (41243351, 41246977),
  (41247762, 41248039),
  (41249160, 41249406),
  (41251691, 41251997),
  (41256038, 41256378),
  (41256784, 41257073),
  (41258372, 41258650),
  (41267642, 41267896),
  (41275933, 41276232),
  (41277187, 41277600)]}

In [8]:
interval_list=[]
[interval_list.extend(x) for x in expanded_tx_exon_dict.itervalues()]
print(interval_list)

[(41196211, 41197919), (41199559, 41199820), (41201037, 41201311), (41202979, 41203234), (41208968, 41209252), (41215249, 41215490), (41215790, 41216068), (41219524, 41219812), (41222844, 41223355), (41226247, 41226638), (41228404, 41228731), (41234320, 41234692), (41242860, 41243149), (41243351, 41246977), (41247762, 41248039), (41249160, 41249406), (41251691, 41251997), (41256038, 41256378), (41256784, 41257073), (41258372, 41258650), (41267642, 41267896), (41275933, 41276232), (41277187, 41277600)]


In [9]:
sorted_by_start=sorted(interval_list,key=lambda x: x[0])
sorted_by_start

[(41196211, 41197919),
 (41199559, 41199820),
 (41201037, 41201311),
 (41202979, 41203234),
 (41208968, 41209252),
 (41215249, 41215490),
 (41215790, 41216068),
 (41219524, 41219812),
 (41222844, 41223355),
 (41226247, 41226638),
 (41228404, 41228731),
 (41234320, 41234692),
 (41242860, 41243149),
 (41243351, 41246977),
 (41247762, 41248039),
 (41249160, 41249406),
 (41251691, 41251997),
 (41256038, 41256378),
 (41256784, 41257073),
 (41258372, 41258650),
 (41267642, 41267896),
 (41275933, 41276232),
 (41277187, 41277600)]

In [10]:
def collapse_intervals(intervals):
    cur_start, cur_stop = intervals[0]
    for next_start, next_stop in intervals[1:]:
        if cur_stop < next_start:
            yield cur_start, cur_stop
            cur_start, cur_stop = next_start, next_stop
        else:
            cur_stop = next_stop
    yield cur_start, cur_stop

In [11]:
test_intervals = [(1,4), (2,5), (5,7), (10,13)] #min set [(1,7), (10,13)]
test_intervals2 = [(1,4), (2,5), (6,7), (10,13), (12,15)] #min set [(1,5), (6,7), (10,15)]
print(list(collapse_intervals(test_intervals)) == [(1,7), (10,13)])
print(list(collapse_intervals(test_intervals2)) == [(1,5), (6,7), (10,15)])

True
True


In [12]:
collapsed_interval_list = list(collapse_intervals(sorted_by_start))
print("Before collapsing we have: " + str(len(interval_list)) + " intervals")
print("After collapsing we have: " + str(len(collapsed_interval_list)) + " intervals")
print("\nOur new interval set:\n")
print(collapsed_interval_list)

Before collapsing we have: 23 intervals
After collapsing we have: 23 intervals

Our new interval set:

[(41196211, 41197919), (41199559, 41199820), (41201037, 41201311), (41202979, 41203234), (41208968, 41209252), (41215249, 41215490), (41215790, 41216068), (41219524, 41219812), (41222844, 41223355), (41226247, 41226638), (41228404, 41228731), (41234320, 41234692), (41242860, 41243149), (41243351, 41246977), (41247762, 41248039), (41249160, 41249406), (41251691, 41251997), (41256038, 41256378), (41256784, 41257073), (41258372, 41258650), (41267642, 41267896), (41275933, 41276232), (41277187, 41277600)]


####  interval list를 엑손 10번으로 교체

In [13]:
collapsed_interval_list=[(41242860, 41243149)]

In [14]:
BASES = ['A', 'G', 'C', 'T']
TWOBASES = [ ''.join(combo) for combo in itertools.permutations(BASES, 2)]
THREEBASES = [ ''.join(combo) for combo in itertools.permutations(BASES, 3)]
print BASES
print TWOBASES
print THREEBASES

['A', 'G', 'C', 'T']
['AG', 'AC', 'AT', 'GA', 'GC', 'GT', 'CA', 'CG', 'CT', 'TA', 'TG', 'TC']
['AGC', 'AGT', 'ACG', 'ACT', 'ATG', 'ATC', 'GAC', 'GAT', 'GCA', 'GCT', 'GTA', 'GTC', 'CAG', 'CAT', 'CGA', 'CGT', 'CTA', 'CTG', 'TAG', 'TAC', 'TGA', 'TGC', 'TCA', 'TCG']


In [15]:
chr7_seq = FastaReader("/NGENEBIO/workflow-dependencies/chr17.fa").__iter__().next()


In [16]:
def get_variants(chr, pos):
    variant_list = []
    #subtract 1 because python lists are 0-indexed; add 3 b/c python list slicing is exclusive of upper index
    ref = chr7_seq[pos-1:pos+3].seq.upper()
    variant_list.extend(get_snps(chr, pos, ref))
    variant_list.extend(get_insertions(chr, pos, ref))
    variant_list.extend(get_deletions(chr, pos, ref))
    return variant_list

def get_snps(chr, pos, ref):
    snp_list = []
    ref = ref[0]
    for base in BASES:
        if base == ref: continue
        snp_list.append({'#CHROM': chr, 'POS': pos, 'REF': ref, 'ALT': base})
    return snp_list

def get_insertions(chr, pos, ref):
    ins_list = []
    ref = ref[0]
    for base in itertools.chain(BASES, random.sample(TWOBASES, 2), random.sample(THREEBASES, 2)):
        ins_list.append({'#CHROM': chr, 'POS': pos,'REF': ref[0], 'ALT': ref[0] + base})
    return ins_list

def get_deletions(chr, pos, ref):
    del_list = []
    del_list.append({'#CHROM': chr, 'POS': pos, 'REF': ref[0:2], 'ALT': ref[0]})
    del_list.append({'#CHROM': chr, 'POS': pos, 'REF': ref[0:3], 'ALT': ref[0]})
    del_list.append({'#CHROM': chr, 'POS': pos, 'REF': ref, 'ALT': ref[0]})
    return del_list

In [17]:
col_names= ['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER','INFO','FORMAT','SAMPLE']
variant_df = pd.DataFrame(columns=col_names)
print variant_df
for interval in collapsed_interval_list:
    pos, stop = interval
    while(pos < stop):
        var_dict = get_variants(7, pos)
        variant_df = variant_df.append(var_dict)
        pos += 1
#hard code in our genotype information
variant_df['FORMAT'] = 'GT'
variant_df['SAMPLE'] = '0/1'

Empty DataFrame
Columns: [#CHROM, POS, ID, REF, ALT, QUAL, FILTER, INFO, FORMAT, SAMPLE]
Index: []


In [18]:
variant_df.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE
0,7.0,41242860.0,NaN,T,A,NaN,NaN,NaN,GT,0/1
1,7.0,41242860.0,NaN,T,G,NaN,NaN,NaN,GT,0/1
2,7.0,41242860.0,NaN,T,C,NaN,NaN,NaN,GT,0/1
3,7.0,41242860.0,NaN,T,TA,NaN,NaN,NaN,GT,0/1
4,7.0,41242860.0,NaN,T,TG,NaN,NaN,NaN,GT,0/1


In [20]:
vcf_header = """##fileformat=VCFv4.1
##fileDate={0}
##source=CFTR_Variant_Project
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
"""
vcf_header = vcf_header.format(datetime.datetime.utcnow().strftime("%Y%m%d"))
'\n'.join([vcf_header, ('\t'.join(variant_df.columns)), '\t'.join(col_names)])

'##fileformat=VCFv4.1\n##fileDate=20161007\n##source=CFTR_Variant_Project\n##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">\n\n#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tSAMPLE\n#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tSAMPLE'

In [21]:
output_file = "brca_test_artifical_variant.vcf"
try:
    os.remove(output_file)
except OSError:
    pass
with open(output_file, 'a') as f:
    f.write(vcf_header)
    variant_df.to_csv(f, sep='\t', na_rep='.', index=False)

In [22]:
store_file="store.h5"
try:
    os.remove(store_file)
except OSError:
    pass
store = pd.HDFStore(store_file)
store['variant_df'] = variant_df

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2881: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']]

  exec(code_obj, self.user_global_ns, self.user_ns)
